<a href="https://colab.research.google.com/github/ExCaLBBR/ExCaLBBR_Projects/blob/main/RaciallyBiasedDecisions/RaciallyBiasedDecisions-Intersectionality_SURG/code/RBD_Intersectionality_IAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd #for dealing with csv import
import os # for joining paths and filenames sensibly
import numpy as np #for the population std
import glob # for finding csv data files
import platform # paths use different dividers on linux vs windows, so we need to test for this

# Reference: 
## IAT data analysis script adapted from https://github.com/tomstafford/IAT

Calculate IAT score from data generated by Robin's PsychoPy script

TQS Feb 2014

Based on

*Greenwald, A. G., Nosek, B. A., & Banaji, M. R. (2003). Understanding and using the implicit association test: I. An improved scoring algorithm. Journal of personality and social psychology, 85(2), 1972-216.*

# Import libraries

# Define util functions

In [2]:
def adjustedmean(RTs,corrs,penalty):
    n=len(corrs) #trials
    n_errors=n-sum(corrs) #errors
    cor_RTs=np.array(corrs)*RTs #sum of correct RTs
    cor_mean=sum(cor_RTs)/sum(corrs)
    
    #mean with errors replaced with penalty value
    return cor_mean+(n_errors*penalty)/n



def exclude_slows(RTs,corrs,slowRT_limit):
    new_rt=[] #holding variables
    new_cr=[]
    for i in range(len(RTs)): #iterate over every item
        if RTs[i] < slowRT_limit: #if it isn't too fast, include RT and corr values
            new_rt.append(RTs[i])
            new_cr.append(corrs[i])
    
    return (new_rt, new_cr)



# Define parameters

In [3]:
penalty=0.600 #penalty - in seconds - for incorrect responses
slowRT_limit=10 #threshold at which slow RTs are discarded
fastRT_limit=0.300 #threshold which defines responses which are "too fast"
fast_prop_limit=0.1 # threshold proportion of "too fast" responses which defines exclusion of ppt


# Load data

In [4]:
iat_bm_path = "https://github.com/ExCaLBBR/ExCaLBBR_Projects/blob/main/RaciallyBiasedDecisions/RaciallyBiasedDecisions-Intersectionality_SURG/data/IAT_BM.csv"
iat_wm_path = "https://github.com/ExCaLBBR/ExCaLBBR_Projects/blob/main/RaciallyBiasedDecisions/RaciallyBiasedDecisions-Intersectionality_SURG/data/IAT_WM.csv"
df_iat_bm = pd.read_csv(iat_bm_path, sep=',', header=0, keep_default_na = True, on_bad_lines='warn')
# df_iat_wm = pd.read_csv(iat_wm_path, on_bad_lines='skip')
df_iat_bm
# df_iat_wm.head(10)


Skipping line 28: expected 1 fields, saw 367
Skipping line 89: expected 1 fields, saw 3
Skipping line 134: expected 1 fields, saw 3
Skipping line 198: expected 1 fields, saw 3
Skipping line 207: expected 1 fields, saw 6
Skipping line 239: expected 1 fields, saw 4
Skipping line 252: expected 1 fields, saw 4
Skipping line 265: expected 1 fields, saw 4
Skipping line 278: expected 1 fields, saw 4
Skipping line 291: expected 1 fields, saw 4
Skipping line 304: expected 1 fields, saw 4
Skipping line 317: expected 1 fields, saw 4
Skipping line 330: expected 1 fields, saw 4
Skipping line 348: expected 1 fields, saw 4
Skipping line 355: expected 1 fields, saw 4
Skipping line 364: expected 1 fields, saw 4
Skipping line 373: expected 1 fields, saw 4
Skipping line 399: expected 1 fields, saw 4
Skipping line 406: expected 1 fields, saw 4
Skipping line 413: expected 1 fields, saw 4
Skipping line 420: expected 1 fields, saw 4
Skipping line 434: expected 1 fields, saw 4
Skipping line 441: expected 1 fi

,<!DOCTYPE html>
0,"<html lang=""en"" data-color-mode=""auto"" data-li..."
1,<head>
2,"<meta charset=""utf-8"">"
3,"<link rel=""dns-prefetch"" href=""https://githu..."
4,"<link rel=""dns-prefetch"" href=""https://avata..."
...,...
1069,</template>
1070,</div>
1071,"<div id=""js-global-screen-reader-notice"" c..."
1072,</body>


In [5]:
IAT_BM = pd.read_csv ('https://github.com/ExCaLBBR/ExCaLBBR_Projects/raw/main/RaciallyBiasedDecisions/RaciallyBiasedDecisions-Intersectionality_SURG/data/IAT_BM.csv', header=0)

In [6]:
IAT_BM

,Unnamed: 0,Participant Private ID,allocator-k3xu,Absolute Reaction Time,Correct,Spreadsheet: ImageLeft,Spreadsheet: ImageRight,Spreadsheet: TextLeft,Spreadsheet: TextRight,Spreadsheet: metadata
0,0,8657252.0,Condition_InIn,2194.605,1.0,bf14_nc.jpg,wf2_nc.jpg,NaN,NaN,practice_images
1,1,8657252.0,Condition_InIn,1177.266,1.0,bf14_nc.jpg,wf2_nc.jpg,NaN,NaN,practice_images
2,2,8657252.0,Condition_InIn,1137.741,1.0,bf14_nc.jpg,wf2_nc.jpg,NaN,NaN,practice_images
3,3,8657252.0,Condition_InIn,790.347,1.0,bf14_nc.jpg,wf2_nc.jpg,NaN,NaN,practice_images
4,4,8657252.0,Condition_InIn,1319.237,1.0,bf14_nc.jpg,wf2_nc.jpg,NaN,NaN,practice_images
...,...,...,...,...,...,...,...,...,...,...
18573,18573,8657651.0,Condition_InIn,435.380,0.0,bf14_nc.jpg,wf2_nc.jpg,Strong,Weak,incongruent
18574,18574,8657651.0,Condition_InIn,1448.240,1.0,bf14_nc.jpg,wf2_nc.jpg,Strong,Weak,incongruent
18575,18575,8657651.0,Condition_InIn,584.960,1.0,bf14_nc.jpg,wf2_nc.jpg,Strong,Weak,incongruent
18576,18576,8657651.0,Condition_InIn,596.780,1.0,bf14_nc.jpg,wf2_nc.jpg,Strong,Weak,incongruent


# Analysis functions

In [ ]:
# files = glob.glob(search_string) #list of data files in the named location

# #now iterate through all data files and calculate the IAT
# for filename in files:

#     #import into python using pandas
#     df = pd.read_csv(filename)
    
#     ## -----------EXTRACT DATA
#     print "extracting data & calculating IAT"
    
#     #extract the stuff we're interested in (n.b i am indexing using the column names defined in the csv)
#     #dropna() drops nans
#     #tolist() converts from series to list
#     corrs=df['key_resp_9.corr'].dropna().tolist()
#     rts=df['key_resp_9.rt'].dropna().tolist()
#     block_length=int(len(corrs)/2)
#     #find order 
#     order=df['order'].tolist()[0]
#     #1 congr then incong
#     #2 incongr then congr
    
#     if order==1:
#         congr_corr=corrs[0:block_length]
#         congr_rts=rts[0:block_length]
#         incon_corr=corrs[block_length:]
#         incon_rts=rts[block_length:]
#     else:
#         congr_corr=corrs[block_length:]
#         congr_rts=rts[block_length:]
#         incon_corr=corrs[0:block_length]
#         incon_rts=rts[0:block_length]
            
    
    
#     ## -----------ANALYSE / CALCULATE
        
#     #1 discard subject if too many fast responses
#     if sum(np.array(congr_rts + incon_rts)<fastRT_limit)>len(congr_rts + incon_rts)*fast_prop_limit:
#         print "excluding subject for " + os.path.basename(filename) + " because too many fast responses"
#     else:
#         #2 Eliminate scores over 10,000 ms
             
#         congr_rts,congr_corr=exclude_slows(congr_rts,congr_corr,slowRT_limit)              
#         incon_rts,incon_corr=exclude_slows(incon_rts,incon_corr,slowRT_limit)
        
#         #3 Calculate pooled std
#         #pooled_std=pooled.std(0) #n-1 std sample std
#         #(Use N not N-1 because this is the whole sample). 
#         #numpy.std is population std
#         pooled=congr_rts + incon_rts #all RTs from both blocks, correct and incorrect
#         pooled_std=np.std(pooled)
        
#         #4 Calculated adjusted means, including the penalty
#         congr_adjmean=adjustedmean(congr_rts,congr_corr,penalty)
#         incon_adjmean=adjustedmean(incon_rts,incon_corr,penalty)
        
#         #5 Calculate the IAT, so that pro-stereotype RTs are a -ve score
#         IAT=(congr_adjmean-incon_adjmean)/pooled_std
        
#         simpleIAT=mean(congr_rts)-mean(incon_rts)        
        
#         print "IAT for " + os.path.basename(filename) + " is : {:+.3f}".format(IAT)
#         print "Mean difference (uncorrected) is {:+.3f}".format(simpleIAT)+" seconds"
        